<a href="https://colab.research.google.com/github/HLCV-23/Inpainting-Detection/blob/Jonathan/classifier_prototype.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! pip install Pillow==8.3.0
! pip install datasets
! pip install transformers[torch]
! pip install evaluate
import os
import cv2
from datasets import load_dataset, Image, Dataset, concatenate_datasets
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms
import torchvision.transforms.functional as TF
import random
import numpy as np
import PIL
! pip show Pillow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 MB 8.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for Pillow: filename=Pillow-8.3.0-cp310-cp310-linux_x86_64.whl size=1335602 sha256=c6886f8597c9737bcb04bfaa5824e59c5d792dc96775796913243441f2089239
  Stored in directory: /root/.cache/pip/wheels/74/03/2c/da7d77d9a20a9d9314883a6ba9d2819f2f1d8218a0f5827b3d
Successfully built Pillow
  Attempting uninstall: Pillow
    Found existing installation: Pillow 8.4.0
    Uninstalling Pillow-8.4.0:
      Successfully uninstalled Pillow-8.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
imageio 2.25.1 requires pillow>=8.3.2, but you have pillow 8.3.0 which is incompatible.
scikit-image 0.19.3 requires pillow!=7.1.0,!=7.1.1,!=8.3.0,>=6.1.0, but you have pillow 8.3.0 which is incompatible.
torchvision 0.15.2+cu118 requires pillow!=8.3.*,>=

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 82.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 99.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 6.1 MB/s eta 0:00:00
Name: Pillow
Version: 8.3.0
Summary: Python Imaging Library (Fork)
Home-page: https://python-pillow.org
Author: Alex Clark (PIL Fork Author)
Author-email: aclark@python-pillow.org
License: HPND
Loc

In [ ]:
device = torch.device("cuda:0")

In [ ]:
"""
TODO: This still creates a data set for segmentation, we need labels instead of masks, Also this is still HuggingFace but I guess we need PyTorch data loader
"""

path = "drive/MyDrive/Prototype-Dataset2"

image_paths = sorted([path + f"/Images/{f}" for f in os.listdir(path + "/Images/")])#.sort(key=lambda x: int(x.removeprefix(path + "/Images/").removesuffix(".png")))
mask_paths = sorted([path + f"/Masks/{f}" for f in os.listdir(path + "/Masks/")])#.sort(key=lambda x: int(x.removeprefix(path + "/Masks/").removesuffix(".png")))

# This directly creates the images
def create_dataset2(image_paths, mask_paths):
  dataset = Dataset.from_dict({
      "image" : image_paths,
      "mask"  : mask_paths
  })
  dataset.cast_column("image", Image())
  #dataset.cast_column("mask", Image())
  return dataset

# This just stores the paths
def create_dataset(image_paths, mask_paths):
  dataset = Dataset.from_dict({"image" : image_paths, "annotation" : mask_paths}).cast_column("image", Image()).cast_column("annotation", Image())
  #dataset = Dataset.from_dict({"image" : image_paths, "annotation" : mask_paths}).cast_column("image", Array3D(shape=(512, 512, 3),dtype="uint32")).cast_column("annotation", Array3D(shape=(512, 512, 1),dtype="uint32"))
  #dataset = Dataset.from_dict({"image" : image_paths, "mask" : mask_paths}).cast_column("image", PIL.Image.open).cast_column("mask", PIL.Image.open)
  return dataset

dataset = create_dataset(image_paths, mask_paths)

print(dataset[0])
print(dataset[1])
print(dataset[2])

dataset = dataset.train_test_split(test_size=0.2)

train = dataset["train"]
test = dataset["test"]
train[0]["image"].show()
train[0]["annotation"].show()


In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
# create dataset with clean examples
NUM_DATA = 15

#augment the data if p_[augmentation] > 0
def transformations(image, mask, p_blur = 0, p_noise = 0, p_compress = 0):
  mask = np.load(mask)
  image = TF.pil_to_tensor(image) / 255
  if random.random() < p_blur:
    image = TF.gaussian_blur(image, 5)
  if random.random() < p_noise:
    noise = 5e-2 * torch.normal(torch.zeros(image.shape), torch.ones(image.shape))
    image += noise
    image = image.clamp(0, 1)
  if random.random() < p_compress:
    # encode image
    result, encimg = cv2.imencode(".jpg",(image * 255).permute(1,2,0).numpy(), [cv2.IMWRITE_JPEG_QUALITY, 10])
    # decode back
    image = cv2.imdecode(encimg, 1)
  return image, mask

# load inpainted images
data_folder = "drive/MyDrive/Scenes_Dataset/Images"
image_paths = [f"{data_folder}/Modified/{f}" for f in os.listdir(f"{data_folder}/Modified/")]
def create_dataset(image_paths, label = 1):
  dataset = Dataset.from_dict({
      "image" : image_paths,
      "label"  : [label for _ in range(len(image_paths))]
  })
  dataset = dataset.cast_column("image", Image())
  return dataset

inpainted_dataset = create_dataset(image_paths)

# load the same amount of clean data as we have inpainted
clean_dataset = create_dataset([f"{data_folder}/Unmodified/{f}" for f in os.listdir(f"{data_folder}/Unmodified")], label = 0)

dataset = concatenate_datasets([clean_dataset, inpainted_dataset]).shuffle()
dataset = dataset.map(lambda dp : {"image" : dp["image"].resize((224, 224)), "label" : dp["label"]}, batch_size = 1000).with_format("torch")
dataset = dataset.filter(lambda example : example["image"].ndim == 3)
# remove corrupted example
dataset = dataset.shuffle().train_test_split(0.1)

Map:   0%|          | 0/3288 [00:00<?, ? examples/s]

In [ ]:
"""
This is from ChatGPT
"""
train_len, val_len = len(dataset["train"]), len(dataset["test"])

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.models import resnet18
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder

# Set device (GPU if available, else CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define transformations for data preprocessing
transform = transforms.Compose([
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# dataset.set_transform(preprocess)


# # Set paths to your training and validation data
# train_path = "/path/to/train/dataset"
# val_path = "/path/to/validation/dataset"

# # Load the training and validation data
# train_data = ImageFolder(train_path, transform=transform)
# val_data = ImageFolder(val_path, transform=transform)

# Create data loaders
batch_size = 64
train_loader = DataLoader(dataset["train"], batch_size=batch_size, shuffle=True)
val_loader = DataLoader(dataset["test"], batch_size=batch_size)

# Load the pre-trained ResNet50 model
model = resnet18(pretrained=True)

# Freeze all the parameters in the pre-trained model
for param in model.parameters():
    param.requires_grad = False

# Modify the last fully connected layer for binary classification
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 2)  # 2 output classes for binary classification

# Send the model to the device
model = model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 50

for epoch in range(num_epochs):
    #train_loader = dataset["train"].iter(batch_size)
    model.train()  # Set the model to training mode

    running_loss = 0.0
    correct_predictions = 0

    for data in train_loader:
        images, labels = data["image"], data["label"]
        images = transform(images.float().reshape(images.shape[0], 3, 224, 224)).to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        # Track training loss and accuracy
        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        correct_predictions += (predicted == labels).sum().item()

    epoch_loss = running_loss / train_len
    epoch_accuracy = correct_predictions / train_len

    print(f"Epoch {epoch + 1}/{num_epochs}:")
    print(f"Training loss: {epoch_loss:.4f}, Training accuracy: {epoch_accuracy:.4f}")

    # Validation loop
    model.eval()  # Set the model to evaluation mode

    running_loss = 0.0
    correct_predictions = 0

    with torch.no_grad():
      #val_loader = dataset["test"].iter(batch_size)
      for data in val_loader:
          images, labels = data["image"], data["label"]
          images = transform(images.float().reshape(images.shape[0], 3, 224, 224))
          images = images.to(device)
          labels = labels.to(device)

          # Forward pass
          outputs = model(images)
          loss = criterion(outputs, labels)

          # Track validation loss and accuracy
          running_loss += loss.item()
          _, predicted = torch.max(outputs.data, 1)
          correct_predictions += (predicted == labels).sum().item()

    epoch_loss = running_loss / val_len
    epoch_accuracy = correct_predictions / val_len

    print(f"Validation loss: {epoch_loss:.4f}, Validation accuracy: {epoch_accuracy:.4f}")

print("Training completed.")


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/50:
Training loss: 0.0118, Training accuracy: 0.4924
Validation loss: 0.0131, Validation accuracy: 0.4909
Epoch 2/50:
Training loss: 0.0114, Training accuracy: 0.5211
Validation loss: 0.0126, Validation accuracy: 0.5793
Epoch 3/50:
Training loss: 0.0111, Training accuracy: 0.5444
Validation loss: 0.0125, Validation accuracy: 0.5488
Epoch 4/50:
Training loss: 0.0109, Training accuracy: 0.5701
Validation loss: 0.0126, Validation accuracy: 0.5366
Epoch 5/50:
Training loss: 0.0108, Training accuracy: 0.5661
Validation loss: 0.0126, Validation accuracy: 0.5732
Epoch 6/50:
Training loss: 0.0108, Training accuracy: 0.5698
Validation loss: 0.0126, Validation accuracy: 0.5732
Epoch 7/50:
Training loss: 0.0107, Training accuracy: 0.5816
Validation loss: 0.0127, Validation accuracy: 0.5671
Epoch 8/50:
Training loss: 0.0106, Training accuracy: 0.5955
Validation loss: 0.0133, Validation accuracy: 0.5518
Epoch 9/50:
Training loss: 0.0107, Training accuracy: 0.5860
Validation loss: 0.0128, Va

KeyboardInterrupt: ignored